# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!python -m pip install scikit-learn --upgrade

    100% |████████████████████████████████| 20.0MB 2.0MB/s eta 0:00:01  5% |█▉                              | 1.1MB 20.0MB/s eta 0:00:01    12% |████                            | 2.5MB 27.9MB/s eta 0:00:01    26% |████████▌                       | 5.3MB 27.8MB/s eta 0:00:01    33% |██████████▋                     | 6.7MB 27.1MB/s eta 0:00:01    47% |███████████████                 | 9.4MB 27.4MB/s eta 0:00:01    67% |█████████████████████▌          | 13.5MB 26.7MB/s eta 0:00:01    74% |███████████████████████▊        | 14.8MB 25.0MB/s eta 0:00:01    87% |████████████████████████████    | 17.4MB 27.0MB/s eta 0:00:01    94% |██████████████████████████████  | 18.8MB 27.6MB/s eta 0:00:01
    100% |████████████████████████████████| 13.4MB 2.9MB/s eta 0:00:01   10% |███▌                            | 1.5MB 22.7MB/s eta 0:00:01    20% |██████▌                         | 2.7MB 23.8MB/s eta 0:00:01    38% |████████████▍                   | 5.2MB 25.6MB/s eta 0:00:01    56% |██████████████████▏   

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertDatabaseName', con=engine)
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Split text into words using NLTK
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
    tokenized = stemmed
    return tokenized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
], verbose=True)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
pipeline.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total= 1.7min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s


[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   35.6s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   28.3s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:    4.8s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   37.1s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   19.4s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   14.3s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   12.3s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   10.3s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   10.2s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   12.1s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   16.6s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   16.1s finished
[Parallel(n_jobs=None)]: Done 100 out of 100 | elap

[Pipeline] ............... (step 3 of 3) Processing clf, total= 8.8min


[Parallel(n_jobs=None)]: Done 100 out of 100 | elapsed:   31.9s finished


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f7ba0423950>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(verbose=True)))],
         verbose=True)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]

In [9]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4988
               request       0.84      0.46      0.60      1153
                 offer       0.00      0.00      0.00        28
           aid_related       0.78      0.68      0.73      2785
          medical_help       0.76      0.06      0.11       535
      medical_products       0.74      0.06      0.11       334
     search_and_rescue       0.62      0.07      0.13       186
              security       0.33      0.01      0.02       124
              military       0.79      0.05      0.10       217
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.42      0.57       427
                  food       0.88      0.63      0.74       747
               shelter       0.82      0.38      0.52       581
              clothing       0.82      0.13      0.23       107
                 money       0.78      

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [50, 100],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, scoring = ['precision', 'recall', 'f1'])

In [12]:
cv.fit(X_train, y_train)

ValueError: 'f1-score' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
best_estimator = cv.best_estimator_
y_pred = best_estimator.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Try adding a feature that cunts the lenght of the text
# Try adding the genre as a feature

In [ ]:
# # With Column Transformer

# X = df.loc[:, ['message', 'genre']]

# message = 'message'
# # message = ['message'] Why thi didn't work?
# genre = ['genre']

# transformer = ColumnTransformer([
#     ('vect', CountVectorizer(tokenizer=tokenize), message),
#     ('tfidf', TfidfTransformer(), message),
#     ('cat', OneHotEncoder(), genre),
# ])

# pipeline = Pipeline([
#     ('t', transformer),
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.